In [2]:
from pathlib import Path
import altair as alt
import duckdb
from utils import (
    loads_cae_db,
    executes_query_duckdb
)

In [3]:
cae_path = Path(
    "../../kids_chances/data/raw/cae_history.txt"
)

destionation_db = Path("../data/cae_db.duckdb")
cae = loads_cae_db(cae_path, destionation_db)

In [4]:
query = """
select distinct(tipo_beneficiario) from cae limit 10000"""
executes_query_duckdb(query, cae)

,tipo_beneficiario
0,EGRESO
1,NUEVO BENEFICIARIO
2,SIN CREDITO POR NO MATRICULA O POR SUSPENSION
3,BENEFICIARIO RENOVANTE
4,DESERCION
5,GRATUIDAD


# Big Table with Summary

In [5]:
query_summary = """

SELECT id_2023 as id,
año_licitacion as año_solicitud,
        genero_registrado as genero,
        dependencia_colegio as colegio,
        region_grupo_familiar as region,
        quintil_ingreso_grupo_familiar as quintil,
        COUNT(CASE WHEN tipo_beneficiario = 'NUEVO BENEFICIARIO' THEN tipo_beneficiario END)
            as total_prestamos,
        COUNT(DISTINCT(nombre_carrera || '-' || nombre_ies)) as cantidad_carreras,
        COUNT(CASE WHEN arancel_solicitado > 0 then arancel_solicitado END)
            as años_financiados,
        COUNT(DISTINCT(CASE WHEN tipo_beneficiario = 'EGRESO' then (tipo_beneficiario || nombre_carrera || nombre_ies) END))
            as total_egresos,
        COUNT(DISTINCT(CASE WHEN tipo_beneficiario = 'DESERCION' then (tipo_beneficiario || nombre_carrera || nombre_ies) END))
            as total_deserciones,
        ROUND(SUM(total_prestado_ajustado),0) as total_prestado,
        ROUND(AVG(porcentaje_financiado),1) as porcentaje_financiado

FROM cae
GROUP BY id, genero, colegio, region, quintil, año_solicitud
"""

In [6]:
final_table_to_query = executes_query_duckdb(query_summary, cae)

In [7]:
final_table_to_query

,id,año_solicitud,genero,colegio,region,quintil,total_prestamos,cantidad_carreras,años_financiados,total_egresos,total_deserciones,total_prestado,porcentaje_financiado
0,433.292,2012,F,Municipal,REGION DEL BIO-BIO,quintil 4,1,3,2,1,0,3607810.0,0.2
1,462.742,2011,F,Municipal,REGION DEL BIO-BIO,quintil 1 y 2,1,1,2,1,0,1946672.0,0.6
2,217.703,2012,M,Municipal,REGION DEL BIO-BIO,quintil 1 y 2,1,1,5,1,0,13780063.0,0.3
3,155.674,2010,M,Particular subvencionado,REGION DEL BIO-BIO,quintil 1 y 2,1,1,3,1,0,5644193.0,1.0
4,475.388,2012,F,Municipal,REGION METROPOLITANA,quintil 5,1,1,2,1,0,4678568.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1290988,283.218,2009,M,Particular subvencionado,REGION METROPOLITANA,quintil 1 y 2,1,1,1,0,0,3865582.0,1.0
1290989,27.513,2009,F,Municipal,REGION METROPOLITANA,quintil 3,1,1,1,0,0,1863273.0,1.0
1290990,390.313,2009,F,Particular pagado,REGION DE LA ARAUCANIA,quintil 4,1,1,1,0,0,3466198.0,1.0
1290991,1.217.886,2024,M,Particular Pagado,REGION DEL BIO-BIO,quintil 1 y 2,1,1,0,0,0,0.0,0.0


---

---

# Aggregated Total Borrowed

In [8]:
DOLAR = 950 # 1 dollar is $950 CLP

query_total_borrowed = f"""
Select 
    region_grupo_familiar as region,
    quintil_ingreso_grupo_familiar as quintil,
    genero_registrado as genero,
    año_operacion as año_financiado,
    round(SUM(total_prestado_ajustado)/{DOLAR}, 0) as total_prestado
FROM cae
GROUP BY region, quintil, genero, año_financiado
"""

total_borrowed = executes_query_duckdb(query_total_borrowed, cae)
total_borrowed

,region,quintil,genero,año_financiado,total_prestado
0,REGION DE LA ARAUCANIA,quintil 1 y 2,M,2016,7379587.0
1,REGION DEL BIO-BIO,quintil 5,M,2016,6164787.0
2,REGION DE TARAPACA,quintil 3,M,2016,676393.0
3,REGION METROPOLITANA,quintil 1 y 2,M,2014,81519606.0
4,REGION DE TARAPACA,quintil 1 y 2,F,2015,2407017.0
...,...,...,...,...,...
2427,REGION DE AYSEN DEL G. C. IBAÑEZ,quintil 1 y 2,M,2007,74132.0
2428,REGION DE AYSEN DEL G. C. IBAÑEZ,quintil 4,M,2008,89921.0
2429,REGION DE LOS RIOS,quintil 3,M,2006,38118.0
2430,REGION DE ARICA Y PARINACOTA,quintil 3,F,2006,19905.0


In [ ]:
# total_borrowed.to_json("../data/total_borrowed.json",
#                        orient="records",
#                        lines=False,
#                        force_ascii=False,
#                        indent=4)

---

---

In [10]:
query_total_granted = f"""
Select 
    region_grupo_familiar as region,
    quintil_ingreso_grupo_familiar as quintil,
    genero_registrado as genero,
    año_operacion as año_financiado,
    COUNT(tipo_beneficiario) as prestamos_entregados
FROM cae
WHERE tipo_beneficiario = 'NUEVO BENEFICIARIO'
GROUP BY region, quintil, genero, año_financiado
"""

total_granted = executes_query_duckdb(query_total_granted, cae)
total_granted

total_granted.to_json("../data/total_cae_granted.json",
                        orient="records",
                        lines=False,
                        force_ascii=False,
                        indent=4)

---

---

In [9]:
h = executes_query_duckdb("select * from cae limit 100000", cae)
h

,id_2023,año_licitacion,año_operacion,genero_registrado,nacionalidad,region_grupo_familiar,quintil_ingreso_grupo_familiar,cambio_ies_carrera,tipo_beneficiario,nivel_estudio_al_ingreso,...,duracion_formal_carrera,contador_años_financiados,dependencia_colegio,arancel_solicitado,arancel_referencia,tipo_titulo,jornada,banco_administrador,porcentaje_financiado,total_prestado_ajustado
0,667.946,2012,2012,F,Chilena,REGION DE VALPARAISO,quintil 4,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,5,1,Particular subvencionado,2043680,2043680,Profesional con Licenciatura,Diurna,ESTADO,1.000000,3449731.840
1,958.754,2017,2017,F,Chilena,REGION DE LA ARAUCANIA,quintil 5,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,7,1,Particular subvencionado,3674581,3674582,Profesional con Licenciatura,Diurna,ESTADO,1.000000,5250976.249
2,668.482,2012,2012,M,Chilena,REGION METROPOLITANA,quintil 5,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,4,1,Particular subvencionado,820429,2109075,Profesional con Licenciatura,Diurna,ESTADO,0.388999,1384884.152
3,676.859,2012,2012,M,Chilena,REGION METROPOLITANA,quintil 1 y 2,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,3,1,Municipal,1113000,1113000,Técnico,Diurna,ESTADO,1.000000,1878744.000
4,677.231,2012,2012,F,Chilena,REGION METROPOLITANA,quintil 1 y 2,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,5,1,Particular subvencionado,1523626,1523626,Profesional,Diurna,ESTADO,1.000000,2571880.688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,398.205,2010,2010,F,Chilena,REGION DEL MAULE,quintil 1 y 2,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,5,1,Municipal,1340000,1340000,Profesional con Licenciatura,Diurna,SCOTIABANK,1.000000,2398600.000
99996,283.103,2008,2008,M,Chilena,REGION DE VALPARAISO,quintil 1 y 2,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,4,1,Estudiante de Colegio o Liceo Particular subve...,1259168,1259168,Profesional,Diurna,SCOTIABANK,1.000000,2287908.256
99997,543.917,2011,2017,M,Chilena,REGION METROPOLITANA,quintil 1 y 2,NO,BENEFICIARIO RENOVANTE,PRIMER AÑO,...,5,6,Municipal,0,1917892,Profesional con Licenciatura,Diurna,ITAU,0.000000,0.000
99998,143.034,2008,2010,F,Chilena,REGION DE COQUIMBO,quintil 1 y 2,NO,BENEFICIARIO RENOVANTE,PRIMER AÑO,...,3,3,Estudiante de Colegio o Liceo Fiscal o Municipal,490000,990000,Técnico,Diurna,ESTADO,0.494949,877100.000


In [10]:
print(len(h["region_grupo_familiar"].unique()), h["region_grupo_familiar"].unique())

16 ['REGION DE VALPARAISO' 'REGION DE LA ARAUCANIA' 'REGION METROPOLITANA'
 'REGION DE LOS LAGOS' "REGION DEL LIB. B. O'HIGGINS" 'REGION DEL BIO-BIO'
 'REGION DE LOS RIOS' 'REGION DE ANTOFAGASTA' 'REGION DE TARAPACA'
 'REGION DEL MAULE' 'REGION DE COQUIMBO' 'REGION DE ARICA Y PARINACOTA'
 'REGION DE AYSEN DEL G. C. IBAÑEZ' 'REGION DE MAGALLANES Y LA ANTARTICA'
 'REGION DE ATACAMA' 'REGION DE ÑUBLE']


In [ ]:
final_table_to_query.to_parquet("../data/summary_table.parquet")